In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install gensim
!pip install xgboost


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
%cd "/content/drive/MyDrive/Colab Notebooks/2023/SBBD_toxic"
!pwd

/content/drive/MyDrive/Colab Notebooks/2023/SBBD_toxic
/content/drive/MyDrive/Colab Notebooks/2023/SBBD_toxic


In [ ]:
#baixe GloVe 
#!wget http://143.107.183.175:22980/download.php?file=embeddings/glove/glove_s300.zip
#!unzip glove_s300.zip
#!wget http://143.107.183.175:22980/download.php?file=embeddings/glove/glove.6B.300d-pt.zip
#!unzip glove.6B.300d-pt.txt

%cd "/content/drive/MyDrive/Colab Notebooks/2023/SBBD_toxic"
!pwd
!unzip '/content/drive/MyDrive/Colab Notebooks/2023/SBBD_toxic/glove_s300.zip'

#glove_file = "glove_s300.txt"
#glove_embeddings = KeyedVectors.load_word2vec_format(glove_file, binary=False)


/content/drive/MyDrive/Colab Notebooks/2023/SBBD_toxic
/content/drive/MyDrive/Colab Notebooks/2023/SBBD_toxic
Archive:  /content/drive/MyDrive/Colab Notebooks/2023/SBBD_toxic/glove_s300.zip
replace glove_s300.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split

# Substitua o 'caminho_do_arquivo_treino.csv' pelo caminho real do seu arquivo CSV de treino
arquivo_csv_treino = '/content/drive/MyDrive/Colab Notebooks/2023/SBBD_toxic/data/hate_speech_data_paula.csv'
dados_treino = pd.read_csv(arquivo_csv_treino)

# Pré-processamento dos dados de treino
X = dados_treino.iloc[:, 0]  # Coluna dos dados preprocessados
y = dados_treino.iloc[:, 1]  # Coluna dos rótulos

# Dividir os dados em conjuntos de treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

train_df = pd.DataFrame({'text': X_train, 'labels': y_train})
eval_df = pd.DataFrame({'text': X_val, 'labels': y_val})

In [ ]:
import pandas as pd
import numpy as np
import re
import string
from gensim.parsing.preprocessing import remove_stopwords
from gensim.models import KeyedVectors
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Input
from tensorflow.keras.optimizers import Adam
import xgboost as xgb

# Função de pré-processamento
def preprocess(text):
    text = text.lower()  # Transformar para minúsculas
    text = remove_stopwords(text)  # Remover stopwords usando Gensim
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)  # Remover pontuação
    return text

# Aplicar pré-processamento nos conjuntos de dados
train_df['text'] = train_df['text'].apply(preprocess)
eval_df['text'] = eval_df['text'].apply(preprocess)

# Carregar word embeddings pré-treinados
#word_embeddings_file = '/content/drive/MyDrive/Colab Notebooks/2023/SBBD_toxic/glove.6B.300d.txt'
#word_embeddings = KeyedVectors.load_word2vec_format(word_embeddings_file, binary=False)

def convert_glove_to_word2vec(glove_file, output_file):
    with open(glove_file, 'r', encoding='utf-8') as f_in:
        lines = f_in.readlines()
        num_vectors = len(lines)
        vector_size = len(lines[0].split()) - 1

    with open(output_file, 'w', encoding='utf-8') as f_out:
        f_out.write(f'{num_vectors} {vector_size}\n')
        for line in lines:
            f_out.write(line)

glove_file = '/content/drive/MyDrive/Colab Notebooks/2023/SBBD_toxic/glove.6B.300d.txt'
word2vec_output_file = '/content/drive/MyDrive/Colab Notebooks/2023/SBBD_toxic/glove.6B.300d-word2vec.txt'

convert_glove_to_word2vec(glove_file, word2vec_output_file)
word_embeddings = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

# Tokenização e padding
max_length = 280
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['text'])
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_df['text'])
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')

eval_sequences = tokenizer.texts_to_sequences(eval_df['text'])
eval_padded = pad_sequences(eval_sequences, maxlen=max_length, padding='post', truncating='post')

# Preparar matriz de embeddings
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 300
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in word_index.items():
    if word in word_embeddings:
        embedding_vector = word_embeddings[word]
        embedding_matrix[i] = embedding_vector

# Definir a arquitetura do modelo LSTM
inputs = Input(shape=(max_length,))
x = Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], trainable=False)(inputs)
x = LSTM(50, return_sequences=True)(x)
x = Dropout(0.5)(x)
x = LSTM(50)(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=outputs)

# Compilar e treinar o modelo
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_padded, train_df['labels'], epochs=10, batch_size=128, validation_split=0.1)

# Remover a última camada (softmax) do modelo para extrair características
feature_extractor = Model(inputs=model.input, outputs=model.layers[-2].output)

#Extrair características do conjunto de teste
train_features = feature_extractor.predict(train_padded)
eval_features = feature_extractor.predict(eval_padded)

#Treinar um classificador XGBoost com as características extraídas
xgb_classifier = xgb.XGBClassifier()
#xgb_classifier.fit(eval_features, eval_df['labels'])
xgb_classifier.fit(train_features, train_df['labels'])

#Avaliar o classificador XGBoost
xgb_preds = xgb_classifier.predict(eval_features)
xgb_f1 = f1_score(eval_df['labels'], xgb_preds)
xgb_precision = precision_score(eval_df['labels'], xgb_preds)
xgb_recall = recall_score(eval_df['labels'], xgb_preds)

print(f'XGBoost F1 Score: {xgb_f1}')
print(f'XGBoost Precision: {xgb_precision}')
print(f'XGBoost Recall: {xgb_recall}')
